In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pandas as pd
import sqlite3
import math

In [5]:
df = pd.read_pickle('magic_dataframe.pkl')
display(df)

,Strategy,Year,Median i-band depth,Mean i-band depth,Std i-band depth,IQR i-band depth
0,baseline_v3.3_10yrs,1,25.252161,25.233240,0.184610,0.235948
1,noroll_v3.3_10yrs,1,25.252161,25.233240,0.184610,0.235948
2,roll_mid1_const_season2v3.3_10yrs,1,25.252161,25.233240,0.184610,0.235948
3,roll_mid1_const_season4v3.3_10yrs,1,25.252161,25.233240,0.184610,0.235948
4,roll_mid1_const_season6v3.3_10yrs,1,25.252161,25.233240,0.184610,0.235948
...,...,...,...,...,...,...
75,roll_mid1_const_season4v3.3_10yrs,10,26.395938,26.369284,0.157892,0.198346
76,roll_mid1_const_season6v3.3_10yrs,10,26.391862,26.363840,0.157892,0.196083
77,roll_mid2_const_season2v3.3_10yrs,10,26.408314,26.377038,0.156602,0.194922
78,roll_mid2_const_season4v3.3_10yrs,10,26.396751,26.370429,0.157705,0.195045


In [6]:
def ilim_to_ngal(mean_ilim, rms_ilim):
    # Takes mean and RMS ilim, returns mean and RMS ngal
    # Let's assume they are arrays - can generalize later.
    # Use ngal(<ilim) = 42.9 * (1-fmask) * 10^(0.359*(ilim-25)) / arcsec^2
    # Assume fmask = 0.1
    # Using this is conservative -- we won't really use things down to the limiting mag, but rather brighter samples.
    fmask = 0.1
    mean_ngal = 42.9 * (1-fmask) * 10**(0.359*(mean_ilim-25))
    # RMS: use error propagation,
    # (Variance of ngal) = (Variance of ilim) * (dngal/dilim)^2
    # (RMS of ngal)^2 = (RMS of ilim)^2 * (dngal/dilim)^2
    # RMS of ngal = (RMS of ilim) * (dngal/dilim)
    # Furthermore, dngal/dilim = ngal * ln(10)*0.359 because of how derivatives of powers work.
    rms_ngal = rms_ilim * mean_ngal * math.log(10) * 0.359
    return mean_ngal, mean_ilim

In [9]:
mean_ngal, rms_ngal = ilim_to_ngal(df['Mean i-band depth'], df['Std i-band depth'])
df['Mean Ngal'] = mean_ngal
df['Std Ngal'] = rms_ngal
df.to_pickle('dataframe_for_federico.pkl')
print(mean_ngal)
print(rms_ngal)

0      46.820182
1      46.820182
2      46.820182
3      46.820182
4      46.820182
         ...    
75    119.749282
76    119.211605
77    120.519244
78    119.862598
79    119.862598
Name: Mean i-band depth, Length: 80, dtype: float64
0     25.233240
1     25.233240
2     25.233240
3     25.233240
4     25.233240
        ...    
75    26.369284
76    26.363840
77    26.377038
78    26.370429
79    26.370429
Name: Mean i-band depth, Length: 80, dtype: float64
